In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import numpy as np

from store_data import DataStore

from utilities import get_month

## Feature Analysis and Engineering

In [2]:
dataset = pd.read_csv('data1/dataset_fairbanks.csv')

In [3]:
dataset.isna().sum()

Unnamed: 0                                         0
Cooling:Electricity [kW](Hourly)                   0
Date/Time                                          0
Electricity:Facility [kW](Hourly)                  0
Fans:Electricity [kW](Hourly)                      0
Gas:Facility [kW](Hourly)                          0
Heating:Electricity [kW](Hourly)                   0
Heating:Gas [kW](Hourly)                           0
InteriorEquipment:Electricity [kW](Hourly)         0
InteriorEquipment:Gas [kW](Hourly)                 0
InteriorLights:Electricity [kW](Hourly)            0
Water Heater:WaterSystems:Gas [kW](Hourly)         0
building_type                                      0
location                                           0
location_id                                        0
building_id                                        0
Date                                               0
Holiday                                       136296
Holiday_code                                  

In [4]:
dataset_expand_holidays = pd.get_dummies(dataset['Holiday'])
dataset = pd.concat([dataset, dataset_expand_holidays], axis=1)
dataset.head(5)

,Unnamed: 0,Cooling:Electricity [kW](Hourly),Date/Time,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Gas:Facility [kW](Hourly),Heating:Electricity [kW](Hourly),Heating:Gas [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),InteriorEquipment:Gas [kW](Hourly),...,Christmas Day,Columbus Day,Independence Day,Labor Day,Martin Luther King Day,Memorial Day,New Years Day,Presidents Day,Thanksgiving,Veterans Day
0,24,0.0,01/01 01:00:00,100.974341,26.200519,64.618438,0.0,62.847188,6.812987,1.75125,...,0,0,0,0,0,0,1,0,0,0
1,25,0.0,01/01 02:00:00,102.514353,27.445885,53.773559,0.0,52.002309,6.812987,1.75125,...,0,0,0,0,0,0,1,0,0,0
2,26,0.0,01/01 03:00:00,93.422126,18.297257,42.808719,0.0,39.893097,6.812987,1.75125,...,0,0,0,0,0,0,1,0,0,0
3,27,0.0,01/01 04:00:00,102.370761,27.445885,71.689463,0.0,69.918213,6.812987,1.75125,...,0,0,0,0,0,0,1,0,0,0
4,28,0.0,01/01 05:00:00,109.981092,28.233191,74.097553,0.0,72.326303,6.812987,1.75125,...,0,0,0,0,0,0,1,0,0,0


In [5]:
dataset = dataset.drop(columns=['Holiday_code', 'Holiday'])
weekday_features = pd.read_csv('data/weekday_features.csv')
weekday_features.head(5)

,Date,Day,is_weekend,is_holiday,Holiday_name
0,01/01,Thursday,0,1,New Years Day
1,01/02,Friday,0,0,NaN
2,01/03,Saturday,1,0,NaN
3,01/04,Sunday,1,0,NaN
4,01/05,Monday,0,0,NaN


In [6]:
dataset = pd.merge(dataset, weekday_features,
                   on='Date', how='inner')
dataset.count()

Unnamed: 0                                    140112
Cooling:Electricity [kW](Hourly)              140112
Date/Time                                     140112
Electricity:Facility [kW](Hourly)             140112
Fans:Electricity [kW](Hourly)                 140112
Gas:Facility [kW](Hourly)                     140112
Heating:Electricity [kW](Hourly)              140112
Heating:Gas [kW](Hourly)                      140112
InteriorEquipment:Electricity [kW](Hourly)    140112
InteriorEquipment:Gas [kW](Hourly)            140112
InteriorLights:Electricity [kW](Hourly)       140112
Water Heater:WaterSystems:Gas [kW](Hourly)    140112
building_type                                 140112
location                                      140112
location_id                                   140112
building_id                                   140112
Date                                          140112
time                                          140112
sunriseTime                                   

### One Hot Encoding

In [7]:
dataset['day_of_week'] = dataset['Day']
dataset_expand_days = pd.get_dummies(dataset['Day'])
dataset = pd.concat([dataset, dataset_expand_days], axis=1)
dataset.head(5)

,Unnamed: 0,Cooling:Electricity [kW](Hourly),Date/Time,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Gas:Facility [kW](Hourly),Heating:Electricity [kW](Hourly),Heating:Gas [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),InteriorEquipment:Gas [kW](Hourly),...,is_holiday,Holiday_name,day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,24,0.0,01/01 01:00:00,100.974341,26.200519,64.618438,0.0,62.847188,6.812987,1.75125,...,1,New Years Day,Thursday,0,0,0,0,1,0,0
1,25,0.0,01/01 02:00:00,102.514353,27.445885,53.773559,0.0,52.002309,6.812987,1.75125,...,1,New Years Day,Thursday,0,0,0,0,1,0,0
2,26,0.0,01/01 03:00:00,93.422126,18.297257,42.808719,0.0,39.893097,6.812987,1.75125,...,1,New Years Day,Thursday,0,0,0,0,1,0,0
3,27,0.0,01/01 04:00:00,102.370761,27.445885,71.689463,0.0,69.918213,6.812987,1.75125,...,1,New Years Day,Thursday,0,0,0,0,1,0,0
4,28,0.0,01/01 05:00:00,109.981092,28.233191,74.097553,0.0,72.326303,6.812987,1.75125,...,1,New Years Day,Thursday,0,0,0,0,1,0,0


In [8]:
dataset.columns

Index(['Unnamed: 0', 'Cooling:Electricity [kW](Hourly)', 'Date/Time',
       'Electricity:Facility [kW](Hourly)', 'Fans:Electricity [kW](Hourly)',
       'Gas:Facility [kW](Hourly)', 'Heating:Electricity [kW](Hourly)',
       'Heating:Gas [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)',
       'InteriorEquipment:Gas [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'Water Heater:WaterSystems:Gas [kW](Hourly)', 'building_type',
       'location', 'location_id', 'building_id', 'Date', 'time', 'sunriseTime',
       'sunsetTime', 'temperatureHigh', 'dewPoint', 'humidity', 'windSpeed',
       'cloudCover', 'Electricity:Facility [kW](Hourly)_lag',
       'Electricity:Facility [kW](Hourly)_future',
       'Electricity:Facility_delta_current_lag', 'Christmas Day',
       'Columbus Day', 'Independence Day', 'Labor Day',
       'Martin Luther King Day', 'Memorial Day', 'New Years Day',
       'Presidents Day', 'Thanksgiving', 'Veterans Day', 'Day', 'is_we

In [9]:
dataset_expand_building_type = pd.get_dummies(dataset['building_type'])
dataset = pd.concat([dataset, dataset_expand_building_type], axis=1)
dataset_expand_location = pd.get_dummies(dataset['location'])
dataset = pd.concat([dataset, dataset_expand_location], axis=1)
dataset.head(5)

,Unnamed: 0,Cooling:Electricity [kW](Hourly),Date/Time,Electricity:Facility [kW](Hourly),Fans:Electricity [kW](Hourly),Gas:Facility [kW](Hourly),Heating:Electricity [kW](Hourly),Heating:Gas [kW](Hourly),InteriorEquipment:Electricity [kW](Hourly),InteriorEquipment:Gas [kW](Hourly),...,RefBldgPrimarySchoolNew,RefBldgQuickServiceRestaurantNew,RefBldgSecondarySchoolNew,RefBldgSmallHotelNew,RefBldgSmallOfficeNew,RefBldgStand-aloneRetailNew,RefBldgStripMallNew,RefBldgSuperMarketNew,RefBldgWarehouseNew,USA_AK_FAIRBANKS
0,24,0.0,01/01 01:00:00,100.974341,26.200519,64.618438,0.0,62.847188,6.812987,1.75125,...,0,0,0,0,0,0,0,1,0,1
1,25,0.0,01/01 02:00:00,102.514353,27.445885,53.773559,0.0,52.002309,6.812987,1.75125,...,0,0,0,0,0,0,0,1,0,1
2,26,0.0,01/01 03:00:00,93.422126,18.297257,42.808719,0.0,39.893097,6.812987,1.75125,...,0,0,0,0,0,0,0,1,0,1
3,27,0.0,01/01 04:00:00,102.370761,27.445885,71.689463,0.0,69.918213,6.812987,1.75125,...,0,0,0,0,0,0,0,1,0,1
4,28,0.0,01/01 05:00:00,109.981092,28.233191,74.097553,0.0,72.326303,6.812987,1.75125,...,0,0,0,0,0,0,0,1,0,1


In [10]:
dataset['month'] = dataset['Date'].apply(get_month)

In [11]:
dataset.to_csv('data1/dataset_fairbanks.csv')

In [12]:
dataset[['Electricity:Facility [kW](Hourly)', 'Electricity:Facility [kW](Hourly)_lag',
       'Electricity:Facility [kW](Hourly)_future',
       'Electricity:Facility_delta_current_lag','Date/Time']].head(50)

,Electricity:Facility [kW](Hourly),Electricity:Facility [kW](Hourly)_lag,Electricity:Facility [kW](Hourly)_future,Electricity:Facility_delta_current_lag,Date/Time
0,100.974341,22.200377,15.004253,78.773964,01/01 01:00:00
1,102.514353,20.673749,15.275587,81.840604,01/01 02:00:00
2,93.422126,22.051864,15.308278,71.370261,01/01 03:00:00
3,102.370761,20.450578,15.425091,81.920183,01/01 04:00:00
4,109.981092,21.914217,15.465380,88.066875,01/01 05:00:00
5,100.414202,20.393177,15.557794,80.021025,01/01 06:00:00
6,135.191711,21.915667,15.596659,113.276044,01/01 07:00:00
7,144.372185,20.429171,15.661208,123.943013,01/01 08:00:00
8,179.829513,21.952760,26.817597,157.876753,01/01 09:00:00
9,152.423660,20.512506,26.817597,131.911154,01/01 10:00:00


In [13]:
"""
data_store = DataStore('35.227.50.121')
data_store.connect_to_database()
data_store.store_data(energystats)
"""

"\ndata_store = DataStore('35.227.50.121')\ndata_store.connect_to_database()\ndata_store.store_data(energystats)\n"